In [ ]:
import numpy as np
from functools import partial
import torch
from lib.timer import Timer
from lib.utils import load_obj, natural_key
from datasets.indoor import IndoorDataset
from datasets.modelnet import get_train_datasets, get_test_datasets
import os,re,sys,json,yaml,random, argparse, torch, pickle
from easydict import EasyDict as edict
from configs.models import architectures
from models.architectures import KPFCNN
import open3d as o3d
import matplotlib.pyplot as plt
# import open3d.visualization.jupyter as o3d_jupyter
import plotly.graph_objects as go

def load_config(path):
    """
    Loads config file:

    Args:
        path (str): path to the config file

    Returns: 
        config (dict): dictionary of the configuration parameters, merge sub_dicts

    """
    with open(path,'r') as f:
        cfg = yaml.safe_load(f)
    
    config = dict()
    for key, value in cfg.items():
        for k,v in value.items():
            config[k] = v

    return config




def get_datasets(config):
    info_train = load_obj(config.train_info)
    train_set = IndoorDataset(info_train,config,data_augmentation=True)
    return train_set



if __name__ == '__main__':
    config = edict(load_config('configs/train/indoor.yaml'))
    # print(config)
    train_set = get_datasets(config)
    # print(train_set[500])
    point = train_set[10]
    x=torch.zeros_like(torch.from_numpy(point['src_pcd'])).cuda().float()
    y=torch.zeros_like(torch.from_numpy(point['tgt_pcd'])).cuda().float()
    color1 = point['src_color1'].cuda()
    color2 = point['tgt_color1'].cuda()
    src1_inds2d=point['src1_inds2d'].cuda()
    src1_inds3d=point['src1_inds3d'].cuda()
    tgt1_inds2d=point['tgt1_inds2d'].cuda()
    tgt1_inds3d=point['tgt1_inds3d'].cuda()
    print(color1.shape)
    print(color2.shape)

    src1_feats = color1[:, src1_inds2d[:, 1], src1_inds2d[:, 0]].transpose(1, 0)
    tgt1_feats = color2[:, tgt1_inds2d[:, 1], tgt1_inds2d[:, 0]].transpose(1, 0)
    print(src1_feats.shape)
    print(tgt1_feats.shape)
    # src1_point_with_2dfeature = torch.cat(
    #                 (src1_feats.transpose(1, 0), torch.ones(src1_feats.shape[1], 1).cuda()), dim=-1).detach()
    x[src1_inds3d, :] = src1_feats.float()
    y[tgt1_inds3d, :] = tgt1_feats.float()
    # print(x)
    # 0/0
    # 创建源点云
    src_pcd = point['src_pcd']
    # 创建目标点云
    tgt_pcd = point['tgt_pcd']


    print("Load a ply point cloud, print it, and render it")
    
    fig1 = go.Figure(
        data=[
            go.Scatter3d(
            x=src_pcd[:,0], y=src_pcd[:,1], z=src_pcd[:,2],
            mode='markers',
            marker=dict(size=1, color=x.cpu().numpy())
    )
    ],
    layout=dict(
        scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    )
    )
    )

    fig1.show() 

    fig2 = go.Figure(
        data=[
            go.Scatter3d(
            x=tgt_pcd[:,0], y=tgt_pcd[:,1], z=tgt_pcd[:,2],
            mode='markers',
            marker=dict(size=1, color=y.cpu().numpy())
    )
    ],
    layout=dict(
        scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    )
    )
    )

    fig2.show()
    

    # 创建 Open3D 点云对象
    pcd = o3d.geometry.PointCloud()

    # 设置点云坐标
    pcd.points = o3d.utility.Vector3dVector(src_pcd)

    # 设置颜色
    pcd.colors = o3d.utility.Vector3dVector(x.cpu().numpy())

    # 保存为 PLY 文件
    o3d.io.write_point_cloud('output.ply', pcd)


